In [219]:
import jieba
import re
import sys ,os
jieba.suggest_freq('韓國瑜', True)
jieba.suggest_freq('兩岸一家親' , True)


sys.setrecursionlimit(1000000)
PUNC = "[0-9a-zA-Z\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）+「」，。）（〔〕／『』:\[\]’：\-《》％‘○]"

text = '陳菊與美國國務院亞太事務代理助理國務卿董雲裳（右）會面。（劉世芳提供）〔記者葛祐豪／高雄報導〕高雄市長陳菊前往美國訪問，今天與美國國務院亞太事務代理助理國務卿董雲裳等人會面；董雲裳強調，台美友誼深厚，也是堅強的夥伴。陳菊在立委劉世芳、賴瑞隆陪同下，今天與美國國務院亞太事務代理助卿董雲裳（Susan Thornton）、經濟暨商業事務首席副助卿麥費特（Brian McFeeters）、亞太事務代理副助卿史墨客（Hanscom Smith）會面。董雲裳回憶說，2000年駐台期間，她在台安醫院生下小女兒，當時父母曾來台探望。前幾天她翻出家人一起環島旅行的相片，對台灣充滿著思念。陳菊告訴董雲裳說，這次訪美主要是到CSIS智庫演講，分享高雄的城市轉型經驗及台灣的民主轉型經驗。陳菊說，她的演講也提到蔡英文總統在內政方面啟動的改革很重要，非成功不可。在兩岸方面，蔡英文溫和、穩健、對中國保持善意，但中國卻沒有正面的回應。她支持蔡總統的政策，也希望美方給予支持。董雲裳表示，台美雙方有互信，更有共同的自由、民主、人權價值，她看到蔡總統戮力改革，積極讓全世界聽到台灣人的聲音。她完全了解改革的困難，也很欽佩蔡總統推動改革的視野。董雲裳認為，台灣只有2300萬人口，卻是美國第11大貿易夥伴。台灣對美國非常重要。台美在安全方面的合作，今後只會更加強，至於台灣參與國際組織，美國一貫支持，也將積極協助提高台灣的國際能見度。副助卿麥費特提醒，美台經濟合作的三大重點：（1）希望美國公司能夠在台公平投資、公平銷售；（2）保護智慧財產權（3）加強高層技術（物聯網、AI等）的合作。董雲裳也提到，北京對台灣的壓力日漸增加，應謀對策，而美國與台灣同為民主國家，應該站在同一陣線。最後，陳菊贈送董雲裳一條排灣族的琉璃珠手鍊。這條「眼睛之珠」，傳說有守護的功用。董雲裳戴上後說，她已經感覺到被保護了。立委劉世芳陪同陳菊與美國國務院亞太事務代理助卿董雲裳（Susan Thornton）等人會面。（劉世芳提供'

In [220]:
text = re.sub(PUNC , "",text)

In [221]:
text_list = list(jieba.cut(text))

In [235]:
def print_rule(my_dict , rules):
    for k,v in rules.items():
        for kk in k:
            print(my_dict.INV_WORD_IDX[kk],end=' ')
        print('->',end=' ')
        for vv in v[0]:
            print(my_dict.INV_WORD_IDX[vv],end=' ')
        print(v[-1])
        
def print_frp(my_dict , patterns):
    for k,v in patterns.items():
        if(len(k) < 2):
            continue
        for kk in k:
            print(my_dict.INV_WORD_IDX[kk],end=' ')
        print(v)

In [255]:
import pickle
from math import log
class shared_dict(object):
    def __init__(self):        
        self.WORD_IDX = {}
        self.INV_WORD_IDX = {}
        self.DF = {}
        self.count = 0       
        self.total_doc = 0 
        self.PUNC = "[0-9a-zA-Z\s+\.\!\/_,$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）+「」，。）（〔〕／『』:\[\]’‘：\-《》％○]"

    
    def add_word(self , word):
        if word in self.WORD_IDX: 
            pass        
        else:
            self.WORD_IDX[word] = self.count
            self.INV_WORD_IDX[self.count] = word
            self.count = self.count + 1
            
    def add_doc(self , doc):
    
        self.total_doc += 1
        word_exist = set()
        
        for word in doc:
            word_exist.add(word)
            self.add_word(word)
            
        for word in word_exist:
            idx = self.WORD_IDX[word]
            if idx in self.DF:
                self.DF[idx] += 1
            else:
                self.DF[idx] = 1
        
    def get_seq(self , word_list , TOPK = 55):
        seq = []
        TF = {}
        tf_idf = {}
        
        total_word = len(word_list)
        
        for word in word_list:
            idx = self.WORD_IDX[word]
            
            if idx in TF:
                TF[idx] += 1
            else:
                TF[idx] = 1
                
        for word in word_list:
            idx = self.WORD_IDX[word]
            tf_idf[idx] = (TF[idx] / total_word) * log( self.total_doc / (self.DF[idx]+1) )
                
                
        sorted_word = list(reversed(sorted(tf_idf.items() , key = lambda d : d[1])))
                
        keep_word = []
                
        for k,v in sorted_word:
            keep_word.append(k)

            if(len(sorted_word) > TOPK):
                keep_word = keep_word[:TOPK]
                        
        return keep_word
                    
    
    def get_transcation_db(self,lines):
        
        # lines should be list of text list
        transcations = []
        for text_list in lines:
            
            seq = self.get_seq(text_list)
            transcations.append(seq)
        
        return transcations
            
    def save_dict(self):
        out_f = open('word_index.pickle','wb')
                    
        pickle.dump(self.WORD_IDX , out_f)
        
        out_f.close()
        
    def load_dict(self):
        in_f = open('word_index.pickle','rb')
        self.WORD_IDX = pickle.load(in_f)
        in_f.close()

        for word , idx in self.WORD_IDX.items():
            self.INV_WORD_IDX[idx] = word
        


In [256]:
ct_dict = shared_dict()
dir_name = '蔡英文_中時'
f_names = os.listdir(dir_name)
text_lists = []
transcations = []

for f_name in f_names:
    f = open(dir_name+'/'+f_name,'r',encoding='utf-8')
    lines = f.readlines()

    
    for text in lines:
        text = re.sub(PUNC , "" , text)
        text_list = list( jieba.cut(text) )
        text_lists.append(text_list)
        ct_dict.add_doc(text_list)

ct_transcations = ct_dict.get_transcation_db(text_lists)


In [257]:
ltn_dict = shared_dict()
dir_name = '蔡英文_自由'
f_names = os.listdir(dir_name)
text_lists = []
transcations = []

for f_name in f_names:
    f = open(dir_name+'/'+f_name,'r',encoding='utf-8')
    lines = f.readlines()

    
    for text in lines:
        text = re.sub(PUNC , "" , text)
        text_list = list( jieba.cut(text) )
        text_lists.append(text_list)
        ltn_dict.add_doc(text_list)

ltn_transcations = ltn_dict.get_transcation_db(text_lists)


In [241]:
ct_num = len(ct_transcations) 
ltn_num = len(ltn_transcations)

In [251]:
thold = 0.01
ct_thold = int ( ct_num * thold )
ltn_thold = int( ltn_num * thold )
ct_thold,ltn_thold

(54, 36)

In [252]:
import pyfpgrowth
ltn_patterns = pyfpgrowth.find_frequent_patterns(ltn_transcations, ltn_thold)
ct_patterns = pyfpgrowth.find_frequent_patterns(ct_transcations,ct_thold)

In [253]:
print_frp(ct_dict , ct_patterns)

非核 家園 61
電廠 深澳 61
美國 旅行 57
賴清德 行政院長 56
美國 美台 59
斷交 布吉納法索 56
多明尼加 斷交 60
蘇 蘇貞昌 57
陳菊 祕書長 54
花蓮 地震 57
美國 中美 58
北農 吳音寧 77
美國 美方 59
教育部 遴選 59
教育部 遴選 校長 54
台大 教育部 遴選 57
台大 教育部 遴選 校長 54
遴選 校長 71
台大 遴選 校長 69
台大 遴選 72
柯文哲 兩岸一家親 72
柯文哲 丁守中 60
兩岸 陸委會 54
大陸 當局 60
美國 美 58
兩岸 善意 68
國民黨 吳敦義 57
大陸 軍演 57
產業 創新 61
兩岸 九二 73
兩岸 九二 共識 57
九二 共識 79
美國 貿易 58
美國 台美 84
賴清德 賴 72
滿意度 滿意 55
民調 滿意 73
民調 滿意度 71
大陸 台海 64
大學 教育部 56
大學 校長 66
大學 台大 66
大學 台大 校長 57
外交 邦交國 60
邦交國 斷交 80
教育部 校長 管中閔 69
台大 教育部 校長 管中閔 64
台大 教育部 管中閔 73
校長 管中閔 96
台大 校長 管中閔 87
台大 管中閔 99
和平 兩岸 85
兩岸 維持 74
兩岸 維持 現狀 56
維持 現狀 85
大陸 統一 63
兩岸 統一 78
大陸 共識 56
兩岸 共識 85
大陸 交流 55
兩岸 交流 80
北市 蘇貞昌 62
北市 新北 72
教育部 校長 104
台大 教育部 校長 94
台大 教育部 110
柯 姚文智 57
台北市長 姚文智 64
民進黨 姚文智 66
柯文哲 姚文智 76
台 美國 69
大陸 台 72
柯文哲 選對會 54
台北市長 選對會 55
人選 選對會 76
民進黨 選對會 81
台北市長 禮讓 59
民進黨 柯 禮讓 68
民進黨 柯 柯文哲 禮讓 58
民進黨 柯文哲 禮讓 74
柯 禮讓 110
柯 柯文哲 禮讓 95
柯文哲 禮讓 126
台獨 現狀 59
兩岸 現狀 100
陳菊 總統府 63
高雄 市長 69
高雄 陳菊 89
台大 校長 129
大陸 川普 54
美國 川普 127
市長 蘇貞昌 78
市長 新北 蘇貞昌 72
新北 蘇貞昌 120
初選 黨內 69
柯 價值 63
柯 柯文哲 價

In [254]:
print_frp(ltn_dict , ltn_patterns)

呂秀蓮 前副總統 37
來源 ‘ 39
新聞 ‘ 39
蔡英文 ‘ 39
新聞 來源 ‘ 39
蔡英文 來源 ‘ 39
新聞 蔡英文 ‘ 39
新聞 蔡英文 來源 ‘ 39
新北 游錫堃 38
柯文哲 兩岸一家親 40
華府 美國 36
民進黨 洪耀福 37
新北 侯友宜 39
吳釗燮 外交部長 37
中國 現狀 39
花蓮 地震 39
台灣 旅行 38
美國 旅行 44
柯文哲 提 40
民進黨 提 45
民進黨 基層 39
前行政院長 蘇貞昌 44
新北 前行政院長 蘇貞昌 41
新北 前行政院長 45
勞基法 修法 40
斷交 布吉納法索 41
時代 力量 41
的 這 38
的 了 36
民進黨 中執會 50
秘書長 接任 38
秘書長 陳菊 接任 36
陳菊 接任 40
民進黨 黨中央 43
北京 美國 36
中國 北京 49
正義 轉型 49
台北市長 民進黨 禮讓 37
台北市長 柯文哲 禮讓 37
柯 民進黨 禮讓 41
柯 柯文哲 民進黨 禮讓 40
柯 柯文哲 禮讓 47
民進黨 禮讓 55
柯文哲 民進黨 禮讓 52
柯文哲 禮讓 61
民進黨 決定 39
關係 台美 37
關係 美國 台美 37
台灣 台美 43
台灣 美國 台美 43
美國 台美 64
台灣 參與 37
民進黨 徵召 51
民進黨 選戰 52
民進黨 呂秀蓮 41
台灣 世界 37
中國 斷交 40
的 個 50
的 是 39
行政院長 賴清德 60
台灣 川普 45
台灣 美國 川普 41
美國 川普 65
柯 價值 36
台灣 價值 40
柯文哲 價值 52
柯文哲 台北市 42
民進黨 台北市 46
民進黨 初選 43
民進黨 議員 39
發展 台灣 36
柯文哲 是否 38
選舉 民進黨 47
中國 外交 41
台灣 外交 45
民進黨 黨內 63
台北市長 選對會 39
選對會 人選 40
柯文哲 選對會 42
民進黨 選對會 72
秘書長 高雄 44
秘書長 高雄 陳菊 44
總統府 秘書長 55
總統府 秘書長 陳菊 47
秘書長 陳菊 67
台灣 經濟 37
柯文哲 姚文智 50
台北市長 姚文智 61
台北市長 民進黨 姚文智 51
民進黨 姚文智 69
市長 高雄 36
高雄 陳菊 75
台灣 合作 36
的 合作 43
民調 民進黨 43
台灣 政府 

In [210]:
a = {
    'a':5,
    'b':6,
    'c':7,
    'd':1
}

In [159]:
zz = sorted(a.items() , key = lambda d : d[1])

In [160]:
for a,b in zz:
    print(a,b)

d 1
a 5
b 6
c 7
